In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import sqlite3

In [2]:
# pickle 저장된 파일 불러오기
import pickle
with open('.train_test.pkl', 'rb') as f:
    x_train, x_test, y_train, y_test = pickle.load(f)
x_train.shape, x_test.shape, y_train.shape

((606, 1, 1000), (152, 1, 1000), (606,))

In [3]:
x_train

array([[[0.01668803, 0.        , 0.03097   , ..., 0.        ,
         0.        , 0.        ]],

       [[0.00750562, 0.        , 0.03714424, ..., 0.        ,
         0.02237014, 0.        ]],

       [[0.04386014, 0.        , 0.01356609, ..., 0.        ,
         0.        , 0.01125551]],

       ...,

       [[0.        , 0.        , 0.06719503, ..., 0.        ,
         0.        , 0.        ]],

       [[0.        , 0.        , 0.10777964, ..., 0.        ,
         0.        , 0.        ]],

       [[0.        , 0.        , 0.06757165, ..., 0.        ,
         0.        , 0.        ]]])

In [4]:
# 저장된 딥러닝 모델 불러오기
from tensorflow.keras.models import load_model
loaded_model = load_model("sports_0802")

In [5]:
pred = loaded_model.predict(x_test)
np.round(pred,2)[:30]

array([[0.05],
       [0.01],
       [0.01],
       [0.  ],
       [0.  ],
       [0.05],
       [0.01],
       [0.9 ],
       [0.94],
       [0.  ],
       [0.2 ],
       [0.  ],
       [0.01],
       [0.14],
       [0.  ],
       [0.  ],
       [0.03],
       [0.  ],
       [0.04],
       [0.06],
       [0.02],
       [0.9 ],
       [0.82],
       [0.03],
       [0.  ],
       [0.  ],
       [0.06],
       [0.  ],
       [0.  ],
       [0.01]], dtype=float32)

In [6]:
# model.accuracy가 안되서 직접 0,1 변환한다음 정확도 계산해보기
pred2=np.where(pred<0.5,0,pred) # 0.5 미만 0
pred2=np.where(pred2>=0.5,1,pred2) # 0.5 이상 1

pred2=pred2.reshape(-1,)
(pred2 == y_test).sum() / y_test.shape[0]

0.8026315789473685

####### 상위 정확도 찾기 #########

In [7]:
one= np.where(pred>0.9)[0]
zero= np.where(pred<0.05)[0]
one, zero

(array([  8,  21,  86, 100, 103, 107, 112, 117, 127, 137], dtype=int64),
 array([  1,   2,   3,   4,   6,   9,  11,  12,  14,  15,  16,  17,  18,
         20,  23,  24,  25,  27,  28,  29,  30,  31,  32,  33,  34,  35,
         36,  37,  38,  40,  43,  44,  49,  54,  55,  56,  57,  58,  63,
         64,  68,  69,  72,  74,  75,  76,  78,  80,  83,  87,  90,  91,
         92,  93,  94,  96,  97,  98,  99, 102, 105, 113, 114, 115, 120,
        123, 124, 126, 129, 133, 134, 135, 138, 140, 144, 145, 147, 148,
        149, 151], dtype=int64))

In [8]:
# 1인거 3개, 0인거 3개 추출
test_one=one[:3]
test_zero=zero[-3:]

test_one, test_zero

(array([ 8, 21, 86], dtype=int64), array([148, 149, 151], dtype=int64))

#####3 확인하기 #####

In [9]:
# PYTHON으로 SQLITE3의 DB에 데이터 담기
dbname='20210726_newDB.db'
def insData(d, tblname='wadiznew'):
    with sqlite3.connect(dbname) as conn:
        cur=conn.cursor()
        sql='select * from '+tblname+' where ncategory="'+d+'"'
        ckkey=cur.execute(sql).fetchall()
        if ckkey:
            return ckkey
        
# df 뽑아내는 함수
def category(i):
    cate=['테크·가전', '패션·잡화', '뷰티', '푸드', '홈리빙', '여행·레저', '스포츠·모빌리티', '공연·컬쳐', 
          '베이비·키즈', '반려동물', '게임·취미', '클래스', '출판', '디자인소품', '소셜·캠페인', '기부·후원', '모임']
    data=insData(cate[i])
    df=pd.DataFrame(data)
    names=['idx','category','name','title','content','supporter','amount','percent','fund','start','end','numofpics','community','key']
    df.columns=names
    encore = "앵콜"
    df['label'] = 0

    for i, title in enumerate(df['title']):
        if title.find(encore) > 0:
            df['label'].iloc[i] = 1
    return df

# 확인용으로 tdf 만들기
# 스포츠모빌리티 선택(6)
df=category(6)
print('data는 ', len(df), '개')
tdf=df.copy()

data는  758 개


C:\Users\pc\anaconda3\envs\py37\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [10]:
# (예측 : 1) 확인해보기 -> 실제 3개 중 2개 (2/3 맞춤)
for i in test_one:
    print(tdf[tdf.index == i][['name','label']])
print('-'*30)   

# (예측 : 0) 확인해보기 -> 실제 3개 중 2개 (2/3 맞춤)
for j in test_zero:
    print(tdf[tdf.index == j][['name','label']])

              name  label
8  타이거다즐러 (올어바웃어스)      0
        name  label
21  주식회사 나노휠      0
        name  label
86  (주)에스앤에프      1
------------------------------
        name  label
148  (주)태서전기      1
                      name  label
149  주식회사 비브로 (VIVRO Inc.)      0
         name  label
151  (주) 핸드허그      0
